## 關於 Optuna
Optuna 是一個專為機器學習設計的自動超參數優化的框架。其最突出的特點是：

- 人性化的定義搜索空間。
- 支援大多數 ML 與 DL 的學習套件。例如: Sklearn、PyTorch、TensorFlow, XGBoost、LightGBM、 CatBoost...等。
- 對對搜索結果提供可解釋性(XAI)。
- 儲存歷史最佳的參數實現平行優化工作。
- 決定並終止不滿足預定義條件的試驗。

In [ ]:
# 首先載入 optuna 套件，如果尚未安裝此套件的的讀者可以參考以下指令進行安裝：

!pip install optuna

## Optuna basics
這裡我們設定一個簡單的目標函式 $(x1+2)^2 + (x2-4)^2$。我們都知道當這個式子 x1=-2, x2=4 時將會有極小值 0。因此我們就用這個簡單的例子透過 Optuna 找出這個函式中極小值所對應的 x1 與 x2 吧。

In [3]:
import optuna

def objective(trial):
    x1 = trial.suggest_float("x1", -5, 5)
    x2 = trial.suggest_float("x2", -5, 5)
    return (x1 + 2) ** 2 + (x2 - 4) ** 2

接著我們來定義一個找出極小值的目標函式 `objective()`。在這個函式中我們將要設定 optuna 可以去尋找的一參數，也就是 x1 與 x2。我們可以透過 optuna 所提供的 `trial` 物件來為我們的超參數設定一組範圍。其中它有一個 `suggest_float` 方法，該方法採用超參數的名稱和範圍來尋找其最佳值。我們以 x1 來舉例：

```
x1 = trial.suggest_float("x1", -5, 5)
```

上面這一段程式在 GridSearch 中可以表示成 `{"x1": np.arange(-5, 5, .1)}`。即表示搜尋過程中我們會從 x1 隨機設定 -5~5 之間的任一浮點數。設定完函式後就可以開始優化了，我們從 optuna 建立一個 `study` 物件，並將 `objective` 函數傳遞給 `study` 的 `optimize` 方法。由於我們的目標是要找出函式中的極小值，因此 `direction` 設為 `minimize`。另外在 `optimize` 方法中我們也可以設定試驗的次數(n_trials)或時間(timeout)。一切就緒後即可開始執行！以下範例是迭代50次並從中找到一組最佳的 x1 與 x2 使其目標函式可以最小化。跑完 50 次後我們可以經由 `study` 變數中得到一組最佳的解。試驗結束後我們可以發現 x1 趨近於 -2 和 x2 趨近於 4。

In [4]:
%%time
# Creating Optuna object and defining its parameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials = 50)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2021-08-04 08:22:50,047] A new study created in memory with name: no-name-d0bdcd8f-3c13-4e67-b82e-d01c7dbbc350
[I 2021-08-04 08:22:50,051] Trial 0 finished with value: 16.94686468110967 and parameters: {'x1': 1.2979908061978929, 'x2': 1.5362383793589443}. Best is trial 0 with value: 16.94686468110967.
[I 2021-08-04 08:22:50,053] Trial 1 finished with value: 54.304611944143886 and parameters: {'x1': -4.5126142507200315, 'x2': -2.9275812208318786}. Best is trial 0 with value: 16.94686468110967.
[I 2021-08-04 08:22:50,054] Trial 2 finished with value: 79.30079467709912 and parameters: {'x1': -4.188218629989622, 'x2': -4.632061972927759}. Best is trial 0 with value: 16.94686468110967.
[I 2021-08-04 08:22:50,056] Trial 3 finished with value: 15.422041642959934 and parameters: {'x1': -3.1889689528087883, 'x2': 0.25722102279379744}. Best is trial 3 with value: 15.422041642959934.
[I 2021-08-04 08:22:50,058] Trial 4 finished with value: 8.355810886963818 and parameters: {'x1': -1.2133304053

Number of finished trials: 50
Best trial parameters: {'x1': -1.8154924755761588, 'x2': 3.9141985823539844}
Best score: 0.04140490983908035
CPU times: user 432 ms, sys: 46.3 ms, total: 478 ms
Wall time: 431 ms


由上述的簡單例子我們可以知道建立一個 optuna 最佳化流程僅需要三步驟：
1. 建立 objective 函式與設定 trial，並回傳 loss。
2. 建立 `create_study()` 物件。
3. 使用 `optimize()` 執行搜尋。


## End-to-end example with XGBoost
我們以 Sklearn 所提供的房價預測資料夾來做範例。此資料集共有 506 筆資料，其中輸入特徵有 13 個其輸出為預測該筆資料的房價。由於想要快速示範如何使用 optuna，因此這裡就不做任何資料 EDA 與前處理。

In [5]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
print('X:',X.shape)
print('y:',y.shape)

X: (506, 13)
y: (506,)


資料集成功被載入後我們就可以建立一個 objective 函式。在這個目標函式中，我們建立了一個小範圍的的 XGBoost 超參數搜索空間。其每一個超參數都會有一個搜索的範圍，可以使用 `suggest_*` 方法設定區間。此方法必須輸入超參數的名稱，以及給予該參數的一組隨機範圍其型態有很多例如：`suggest_int`、`suggest_discrete_uniform`、`suggest_float`...等。更多詳細的內容可以從[官方文件](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html)取得。或是也可以參考官方在 [GitHub](https://github.com/optuna/optuna-examples/tree/main/xgboost) 上對於 XGBoost 的使用範例。

In [6]:
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def objective(trial, X=X, y=y):
    """
    A function to train a model using different hyperparamerters combinations provided by Optuna. 
    Log loss of validation data predictions is returned to estimate hyperparameters effectiveness.
    """
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4)

    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000, 100),
        'eta': trial.suggest_float("eta", 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'gamma': trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
        'grow_policy': trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
    }

    reg = xgb.XGBRegressor(**params)
    reg.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)], eval_metric='rmse',
            verbose=False)
    return mean_squared_error(y_valid, reg.predict(X_valid), squared=False)

In [13]:
%%time
# Creating Optuna ob＄＄ject and defining its parameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials = 10)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2021-08-04 08:31:02,400] A new study created in memory with name: no-name-4eaf3c4d-e434-4dfd-9b0e-1a4a678863b6
[I 2021-08-04 08:31:02,966] Trial 0 finished with value: 23.43004811155196 and parameters: {'max_depth': 14, 'subsample': 0.8998831844318185, 'n_estimators': 700, 'eta': 5.2744054065145944e-05, 'alpha': 1.9595998907215243e-06, 'lambda': 3.8886724800128816e-07, 'gamma': 0.0003401624372181507, 'min_child_weight': 10, 'grow_policy': 'depthwise', 'colsample_bytree': 0.4060504769012863}. Best is trial 0 with value: 23.43004811155196.
[I 2021-08-04 08:31:04,536] Trial 1 finished with value: 3.623483113631351 and parameters: {'max_depth': 8, 'subsample': 0.9933787227246718, 'n_estimators': 1800, 'eta': 0.0027570634364581445, 'alpha': 3.2415102097910046e-06, 'lambda': 3.9016314675928876e-05, 'gamma': 0.10087309991295847, 'min_child_weight': 9, 'grow_policy': 'lossguide', 'colsample_bytree': 0.5554347428204337}. Best is trial 1 with value: 3.623483113631351.
[I 2021-08-04 08:31:05,5

Number of finished trials: 10
Best trial parameters: {'max_depth': 14, 'subsample': 0.27261445097806325, 'n_estimators': 1200, 'eta': 0.005969814699373777, 'alpha': 0.00022127041330143656, 'lambda': 3.946866636631199e-07, 'gamma': 9.812798292197102e-07, 'min_child_weight': 6, 'grow_policy': 'depthwise', 'colsample_bytree': 0.22501880363625518}
Best score: 3.075839347690606
CPU times: user 30 s, sys: 535 ms, total: 30.5 s
Wall time: 9.61 s


Optuna 預設的超參數搜尋方法能有效地在短時間內往最佳的方向去尋找一組適合的參數。與 GridSearch 相比原本可能需要數小時的搜索空間在短短的幾分鐘內就可以獲得不錯的經果。並且有效的降低 loss。除了回歸問題 Optuna 也能對分類問題進行超參數搜尋，官方的 [GitHub](https://github.com/optuna/optuna-examples) 也有提供各種不同機器學習框架的寫法。

##  Optuna如何採樣參數？
TPESampler 為預設的超參數採樣器。它試圖透過提高最後一次試驗的分數來對超參數候選者進行採樣。除此之外 Optuna 提供了以下這幾個參數採樣的方式:
- `GridSampler`: 與 Sklearn 的 `GridSearch` 採樣方式相同。使用此方法時建議不要設定太大的範圍。
- `RandomSampler`: 與 Sklearn 的 `RandomizedGridSearch` 採樣方式相同。
- `TPESampler`: 全名 Tree-structured Parzen Estimator sampler。預設採樣方式。
- `CmaEsSampler`: 基於 CMA ES 演算算法的採樣器 (不支援類別型的超參數).

如果需要替換採樣參數的方式可以參考以下程式。

In [ ]:
from optuna.samplers import CmaEsSampler, RandomSampler

# Study with a random sampler
study = optuna.create_study(sampler=RandomSampler(seed=1121218))

# Study with a CMA ES sampler
study = optuna.create_study(sampler=CmaEsSampler(seed=1121218))

## Optuna 視覺化分析
Optuna 在同時也提供了視覺化的套件:
- plot_optimization_history (視覺化優化的過程)
- plot_intermediate_values (視覺化學習的曲線)
- plot_parallel_coordinate (視覺化高維度中參數間的彼此關係)
- plot_contour (視覺化參數間的彼此關係)
- plot_slice (視覺化個別參數)
- plot_param_importances (參數對模型的重要程度)
- plot_edf (視覺化驗分佈函數)


延續上面的範例我們來視覺化展示 Optuna 搜尋的過程與結果。首先我們來繪製 `study` 的優化歷史過程。

In [14]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

這張圖告訴我們，Optuna 只經過幾次試驗就使分數收斂到最小值。接下來，讓我們繪製超參數重要性：

In [12]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

從這張圖我們可以發現 eta(learning_rate) 學習速率是最為重要的。此外 grow_policy 與 lambda 對減少 loss 上無太大幫助。因此在下一次執行試驗的時候可以考慮將無用的參數移除，並將重要的超參數範圍加大取得更好的搜索結果。其他的使用方法可以 [參考](https://optuna.readthedocs.io/en/stable/reference/visualization/index.html) 官方的說明文件。